In [1]:
import pandas as pd
from pandas import DataFrame as df
import numpy as np
import time
####################################
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import re as re
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [2]:
#1 Remove all symbols
def remove_symbs(text):
    html = re.compile(r'[^\w]')
    return html.sub(r" ", text)

#2 Remove > -> output
def afterfix(text):
    there = re.compile(' › ')
    return there.sub('/', text)

#3 Remove LLC
def removeLLC(text):
    LLC = re.compile(' LLC|LLC | L L C')
    return LLC.sub(' ', text)

#4 Remove LP
def removeLP(text):
    LP = re.compile('LP|L P')
    return LP.sub(' ', text)

#5 Remove B2B
def removeB2B(text):
    B2B = re.compile('B2B')
    return B2B.sub(' ', text)

#6 Remove Org
def removeOrg(text):
    Org = re.compile('Org|ORG|ORG ID|ID')
    return Org.sub(' ', text)

#7 Remove /.../ -> Output
def removedots(text):
    dots = re.compile("[\.]"*3)
    return dots.sub(' ', text)

#8 Remove everything thats starts with 00 (Org ID)
def splitorgid(text):
    text = text.split('00', 1)[0]
    return text

#9 Special removals
def remove2U(text):
    U = re.compile('NewOrg6 |Chartreuse')
    return U.sub(' ', text)

In [3]:
s = pd.read_csv('XXX.csv')
b = []

In [4]:
# Remove duplicates and apply functions
s = s.drop_duplicates()
s.reset_index(drop=True, inplace=True)
s["Customer Name"] = s['Customer Name'].map(lambda x: remove_symbs(x)) #1 --> #1
s["Customer Name"] = s['Customer Name'].map(lambda x: removeLLC(x)) #2 --> #3
s["Customer Name"] = s['Customer Name'].map(lambda x: removeLP(x)) #3 --> #4
s["Customer Name"] = s['Customer Name'].map(lambda x: removeB2B(x)) #4 --> #5
s["Customer Name"] = s['Customer Name'].map(lambda x: removeOrg(x)) #5 --> #6
s["Customer Name"] = s['Customer Name'].map(lambda x: remove2U(x)) #6 --> #9
s["Customer Name"] = s['Customer Name'].map(lambda x: splitorgid(x)) #7 --> #8

In [5]:
s

,Customer Name
0,Linkedin
1,Mercedes Benz
2,Pringles
3,Microsoft


In [6]:
xpaths = ['//*[@id="rso"]/div[1]/div/div[1]/div/a/div/cite',
         '//*[@id="rso"]/div[1]/div/div/div/div/div/div/div[1]/a/div/cite',
         '//*[@id="rso"]/div[1]/div/div/div/div/div/div[1]/a/div/cite',
         '//*[@id="rso"]/div[1]/div/div/div[1]/div/a/div/cite',
         '//*[@id="rso"]/div[3]/div/div[1]/div/a/div/cite',
         '//*[@id="rso"]/div[2]/div/div/div/div/div/div[1]/a/div/cite',
         '//*[@id="kp-wp-tab-overview"]/div[1]/div/div/div/div/div/div/div/div/div/div/div/div[1]/a/div/cite',
         '//*[@id="rso"]/div[2]/div/div/div/div/div/div/div[1]/a/div/cite',
         '//*[@id="rso"]/div[3]/div/div[1]/div/div/div[1]/div/a/div/cite',
         '//*[@id="rso"]/div[3]/div/div/div[1]/div/a/div/cite',
         '//*[@id="rso"]/div[2]/div/div/div[1]/div/a/div/cite',
         '//*[@id="rso"]/div[1]/div/div[1]/div/a/div/cite',
         '//*[@id="kp-wp-tab-overview"]/div[1]/div/div/div/div/div/div/div/div[1]/div/a/div/cite',
         '//*[@id="kp-wp-tab-overview"]/div[2]/div/div/div/div/div/div/div/div/div/div/div/div[1]/a/div/cite',
         '//*[@id="rso"]/div[3]/div/div/div/div/div/div/div[1]/a/div/cite',
         '//*[@id="rso"]/div[4]/div/div[1]/div/a/div/cite',
         '//*[@id="rso"]/div[4]/div/div[1]/div/a/div/cite',
         '//*[@id="kp-wp-tab-overview"]/div[2]/div/div/div/div/div/div/div[1]/div/div/div[1]/div/a/div/cite',
         '//*[@id="kp-wp-tab-overview"]/div[1]/div/div/div/div/div/div/div[1]/div/div/div[1]/div/a/div/cite',
         '//*[@id="rso"]/div[4]/div/div[1]/div/div/div[1]/div/a/div/cite',
        '//*[@id="kp-wp-tab-overview"]/div[1]/div/div/div/div/div/div/div[1]/div/div/div/div/div/div/div[1]/a/div/cite'
         ]

In [12]:
#accessing Chromedriver
browser = webdriver.Chrome(executable_path = 'PATH\chromedriver.exe')

#Open login page
browser.get('https://www.google.com')
#Allow all cookies
browser.find_element(By.XPATH, value='//*[@id="L2AGLb"]').click()

# Run 1
t = s.iloc[0]['Customer Name'] #Take first entry
# Find the search bar
elementID = browser.find_element(By.XPATH, value='/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
# Type first entry
elementID.send_keys(t)
elementID.send_keys(Keys.ENTER)
time.sleep(7)
# Take the first link
for xpath in xpaths:
    try:
        link = browser.find_element(By.XPATH, value=xpath).text
    except:
        pass
b.append(link)

for i in range(1,len(s)):
    t = s.iloc[i]['Customer Name']
    elementID = browser.find_element(By.XPATH, value='//*[@id="tsf"]/div[1]/div[1]/div[2]/div/div[2]/input')
    elementID.clear()
    elementID.send_keys(t)
    elementID.send_keys(Keys.ENTER)
    time.sleep(7)
    # Get the first link in different scenarios.
    for xpath in xpaths:
        try:
            link =  browser.find_element(By.XPATH, value=xpath).text
        except:         
            pass
        
    b.append(link)

browser.quit()

<ipython-input-12-8b77d4087add>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path = 'C:/Users\zaid_\Downloads\chromedriver.exe')


In [13]:
# Create the output
output= df(b)
# Clean the output
for i in range(0,len(output)):
    output[0][i] = afterfix(str(output[0][i])) #1 --> #2
    output[0][i] = removedots(output[0][i]) #2 --> #7

In [14]:
# Save CSV
List = s
List = List.join(output)
List.to_csv('XXX - Output.csv')

In [15]:
output

,0
0,https://uk.linkedin.com
1,https://www.mercedes-benz.com/
2,https://www.pringles.com/
3,https://www.pringles.com/
